# Demographics

Starsim can include detailed population dynamics through its demographics modules. These modules handle births, deaths, and pregnancy to model realistic population changes over time. By default, Starsim simulations have a fixed population size, but adding demographics allows the population to grow, shrink, and change age structure naturally.

Demographics modules are essential for modeling disease transmission over longer time periods, understanding generational effects, and capturing realistic population dynamics.

## Simple usage: Comparing simulations with and without demographics

Let's start by comparing two identical simulations - one with demographics enabled and one without:

```python
import starsim as ss
import matplotlib.pyplot as plt

# Create simulation without demographics (default)
sim_no_demog = ss.Sim(
    diseases='sis',
    networks='mfnet', 
    dur=30,
    label='No demographics'
)

# Create simulation with demographics enabled
sim_with_demog = ss.Sim(
    diseases='sis',
    networks='mfnet',
    demographics=True,  # Enable default demographics
    dur=30,
    label='With demographics'
)

# Run both simulations
sims = ss.parallel(sim_no_demog, sim_with_demog)

# Plot population sizes
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
for sim in sims:
    plt.plot(sim.timevec, sim.results.n_alive, label=sim.label, linewidth=2)
plt.title('Population size over time')
plt.xlabel('Year')
plt.ylabel('Number of people')
plt.legend()

plt.subplot(1, 2, 2)
for sim in sims:
    plt.plot(sim.timevec, sim.results.sis.n_infected, label=sim.label, linewidth=2)
plt.title('SIS infections over time')
plt.xlabel('Year')
plt.ylabel('Number infected')
plt.legend()

plt.tight_layout()
plt.show()
```

Notice how the simulation with demographics shows:
- Growing population over time due to births
- Fluctuating infection numbers that follow population dynamics
- More realistic disease transmission patterns

## Using birth and death rates

You can control demographics using simple birth and death rates:

```python
# Simulation with custom birth and death rates
sim_custom = ss.Sim(
    diseases='sis',
    networks='mfnet',
    birth_rate=ss.peryear(25),  # 25 births per 1000 people per year
    death_rate=ss.peryear(12),  # 12 deaths per 1000 people per year
    dur=50,
    label='Custom demographics'
)

sim_custom.run()

# Plot demographics results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(sim_custom.timevec, sim_custom.results.n_alive, linewidth=2)
plt.title('Total population')
plt.xlabel('Year')
plt.ylabel('Number of people')

plt.subplot(2, 2, 2) 
plt.plot(sim_custom.timevec, sim_custom.results.births.new, linewidth=2, label='Births')
plt.plot(sim_custom.timevec, sim_custom.results.deaths.new, linewidth=2, label='Deaths')
plt.title('Births and deaths per timestep')
plt.xlabel('Year')
plt.ylabel('Number per timestep')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(sim_custom.timevec, sim_custom.results.births.cbr, linewidth=2)
plt.title('Crude birth rate')
plt.xlabel('Year')
plt.ylabel('Births per 1000 per year')

plt.subplot(2, 2, 4)
plt.plot(sim_custom.timevec, sim_custom.results.deaths.cmr, linewidth=2)
plt.title('Crude mortality rate')
plt.xlabel('Year')
plt.ylabel('Deaths per 1000 per year')

plt.tight_layout()
plt.show()

print(f"Population grew from {sim_custom.results.n_alive[0]:,.0f} to {sim_custom.results.n_alive[-1]:,.0f}")
```

## Advanced usage: Pregnancy and births

For more detailed modeling, you can use the `Pregnancy` module instead of simple `Births`. The pregnancy module models:
- Age-specific fertility rates
- Pregnancy duration and outcomes
- Maternal and neonatal mortality
- Mother-to-child transmission pathways

```python
import pandas as pd

# Create age-specific fertility data
fertility_data = pd.DataFrame({
    'AgeGrpStart': [15, 20, 25, 30, 35, 40, 45],
    'ASFR': [0.05, 0.15, 0.20, 0.15, 0.10, 0.05, 0.01]  # Age-specific fertility rates
})

# Create a pregnancy module with custom parameters
pregnancy = ss.Pregnancy(
    fertility_rate=fertility_data,
    dur_pregnancy=ss.years(0.75),           # 9 months pregnancy
    dur_postpartum=ss.years(0.5),           # 6 months postpartum
    p_maternal_death=ss.bernoulli(0.001),   # 0.1% maternal mortality
    p_neonatal_death=ss.bernoulli(0.02),    # 2% neonatal mortality
    min_age=15,
    max_age=50,
)

# Create deaths module
deaths = ss.Deaths(death_rate=ss.peryear(8))  # 8 deaths per 1000 per year

# Create simulation with pregnancy and deaths
sim_pregnancy = ss.Sim(
    diseases='hiv',  # Use HIV to show mother-to-child transmission
    networks='mfnet',
    demographics=[pregnancy, deaths],
    dur=30,
    label='Pregnancy model'
)

sim_pregnancy.run()

# Plot pregnancy-specific results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(sim_pregnancy.timevec, sim_pregnancy.results.n_alive, linewidth=2)
plt.title('Population with pregnancy model')
plt.xlabel('Year')
plt.ylabel('Number of people')

plt.subplot(2, 2, 2)
plt.plot(sim_pregnancy.timevec, sim_pregnancy.results.pregnancy.pregnant, linewidth=2, label='Pregnant')
plt.plot(sim_pregnancy.timevec, sim_pregnancy.results.pregnancy.postpartum, linewidth=2, label='Postpartum')
plt.title('Pregnancy states')
plt.xlabel('Year')
plt.ylabel('Number of people')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(sim_pregnancy.timevec, sim_pregnancy.results.pregnancy.new, linewidth=2, label='Births')
plt.plot(sim_pregnancy.timevec, sim_pregnancy.results.deaths.new, linewidth=2, label='Deaths')
plt.title('Births and deaths')
plt.xlabel('Year')
plt.ylabel('Number per timestep')
plt.legend()

plt.subplot(2, 2, 4)
age_bins = sim_pregnancy.people.age.hist()
plt.bar(range(len(age_bins)), age_bins, alpha=0.7)
plt.title('Final age distribution')
plt.xlabel('Age bin')
plt.ylabel('Number of people')

plt.tight_layout()
plt.show()

print(f"Total pregnancies: {sim_pregnancy.results.pregnancy.cumulative[-1]:,.0f}")
print(f"Final population: {sim_pregnancy.results.n_alive[-1]:,.0f}")
```

## Comparing pregnancy vs. simple births

Let's compare the two approaches to see their differences:

```python
# Simple births model
sim_births = ss.Sim(
    diseases='hiv',
    networks='mfnet',
    birth_rate=ss.peryear(20),
    death_rate=ss.peryear(8),
    dur=30,
    label='Simple births'
)

# Both simulations
sims_compare = ss.parallel(sim_pregnancy, sim_births)

# Compare results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
for sim in sims_compare:
    plt.plot(sim.timevec, sim.results.n_alive, label=sim.label, linewidth=2)
plt.title('Population comparison')
plt.xlabel('Year')
plt.ylabel('Number of people')
plt.legend()

plt.subplot(1, 2, 2)
for sim in sims_compare:
    if hasattr(sim.results, 'pregnancy'):
        plt.plot(sim.timevec, sim.results.pregnancy.new, label=f'{sim.label} births', linewidth=2)
    else:
        plt.plot(sim.timevec, sim.results.births.new, label=f'{sim.label} births', linewidth=2)
plt.title('Birth patterns')
plt.xlabel('Year')
plt.ylabel('Births per timestep')
plt.legend()

plt.tight_layout()
plt.show()
```

## Key concepts

**Demographics modules** enable realistic population dynamics in Starsim:
- **Births**: Simple constant or time-varying birth rates
- **Deaths**: Background mortality separate from disease deaths  
- **Pregnancy**: Detailed pregnancy modeling with maternal/neonatal outcomes

**Key parameters:**
- Use `demographics=True` for default birth/death rates
- Use `birth_rate` and `death_rate` for custom constant rates
- Use `ss.Pregnancy()` for age-specific fertility and pregnancy modeling
- Demographics automatically enables aging (`use_aging=True`)

**Integration with diseases:**
- Pregnancy module supports mother-to-child transmission
- Deaths from demographics are separate from disease mortality
- Newborns enter the population as susceptible to diseases

Demographics are essential for long-term simulations, generational studies, and realistic population modeling in epidemiological contexts.